In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 공통설정

In [1]:
import os
import re
import json
import glob
import gc
import traceback

import pandas as pd
from typing import List, Dict, Any
from datasets import Dataset

from dotenv import load_dotenv
load_dotenv()

from ragas import evaluate

# ---- AnswerRelevancy / Faithfulness import (ragas 버전 차이 대응) ----
try:
    from ragas.metrics._answer_relevancy import AnswerRelevancy
except Exception:
    from ragas.metrics import answer_relevancy as AnswerRelevancy

try:
    from ragas.metrics._faithfulness import Faithfulness
except Exception:
    from ragas.metrics import faithfulness as Faithfulness

from langchain_upstage import ChatUpstage, UpstageEmbeddings
from chatbot_app.modules.rag_module import create_pipeline

from ragas.run_config import RunConfig
rc = RunConfig(max_workers=1, timeout=180)

# 저장명
# PROMPT_TAG = "prompt_v01"

# -----------------------------
# LLM / Embeddings (평가용)
# -----------------------------
# ragas가 LangChain LLM wrapper를 요구하는 버전이 있어서 안전하게 래핑 시도
try:
    from ragas.llms import LangchainLLMWrapper
    ragas_llm = LangchainLLMWrapper(ChatUpstage(model="solar-pro2", temperature=0))
except Exception:
    ragas_llm = ChatUpstage(model="solar-pro2", temperature=0)

ragas_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
upstage_embedding = UpstageEmbeddings(model="solar-embedding-1-large")

print(f"ragas_llm={type(ragas_llm)}")
print(f"ragas_embeddings={type(ragas_embeddings)}")

# -----------------------------
# Metrics
# -----------------------------
def _metric_instance(m):
    # m이 클래스면 m()로 인스턴스 만들고,
    # 이미 인스턴스/객체면 그대로 반환
    return m() if callable(m) else m


AR_METRIC = _metric_instance(AnswerRelevancy)
F_METRIC = _metric_instance(Faithfulness)

OUT_DIR = "C:\ai\source\chatbot_app"
os.makedirs(OUT_DIR, exist_ok=True)

# -----------------------------
# 몽키패치: AnswerRelevancy 커널 크래시(세그폴트) 회피용
# - AnswerRelevancy 내부 cosine similarity 계산이 numpy/BLAS 경로를 타면
#   Windows 환경에서 드물게 하드 크래시가 발생할 수 있어,
#   similarity 계산만 "완전 파이썬" 구현으로 우회합니다.
# - 또한 providers 제약(Upstage: n must be 1) 때문에 strictness=1을 강제합니다.
# -----------------------------
import math
import numpy as np

import ragas.metrics._answer_relevance as ar

def _cos_sim(u, v, eps=1e-8):
    # numpy dot/BLAS 안 쓰는 순수 파이썬 코사인
    su = 0.0
    sv = 0.0
    s  = 0.0
    for a, b in zip(u, v):
        af = float(a); bf = float(b)
        s  += af * bf
        su += af * af
        sv += bf * bf
    return s / (math.sqrt(su) * math.sqrt(sv) + eps)

def safe_calculate_similarity(self, question: str, generated_questions: list[str]):
    """
    ragas AnswerRelevancy/ResponseRelevancy가 기대하는 시그니처:
      (self, question: str, generated_questions: list[str]) -> array-like with .mean()
    """
    # 1) metric이 들고 있는 embeddings 잡기
    emb = None
    for name in ["embeddings", "_embeddings", "embedding", "_embedding"]:
        if hasattr(self, name):
            emb = getattr(self, name)
            if emb is not None:
                break

    if emb is None:
        raise RuntimeError("AnswerRelevancy metric에 embeddings가 세팅되지 않았습니다.")

    # 2) 임베딩 함수 선택 (sync만)
    def embed_one(text: str):
        if hasattr(emb, "embed_query"):
            return emb.embed_query(text)
        if hasattr(emb, "embed_documents"):
            return emb.embed_documents([text])[0]
        raise RuntimeError("embeddings 객체에 embed_query/embed_documents가 없습니다(동기 메서드 필요).")

    def embed_many(texts: list[str]):
        if len(texts) == 0:
            return []
        if hasattr(emb, "embed_documents"):
            return emb.embed_documents(texts)
        # embed_query만 있으면 루프
        if hasattr(emb, "embed_query"):
            return [emb.embed_query(t) for t in texts]
        raise RuntimeError("embeddings 객체에 embed_query/embed_documents가 없습니다(동기 메서드 필요).")

    # 3) 문자열 -> 벡터
    qv = embed_one(question)
    gvs = embed_many(generated_questions)

    if not gvs:
        return np.array([0.0], dtype="float32")

    # 4) 코사인 유사도 (BLAS 안 탐)
    sims = [_cos_sim(qv, gv) for gv in gvs]

    # ragas는 뒤에서 cosine_sim.mean()을 호출하므로 numpy array로 반환
    return np.array(sims, dtype="float32")

# ✅ 클래스 메서드를 덮어써야 함 (모듈 함수 패치가 아님)
ar.ResponseRelevancy.calculate_similarity = safe_calculate_similarity
ar.AnswerRelevancy.calculate_similarity = safe_calculate_similarity

# 1) 모듈 레벨 함수가 있다면 __code__ 교체
if ar is not None and hasattr(ar, "calculate_similarity"):
    ar.calculate_similarity.__code__ = _cos_sim.__code__
    ar.calculate_similarity.__defaults__ = _cos_sim.__defaults__
    print("✅ patched ar.calculate_similarity (__code__)")
elif ar is not None:
    # 2) 모듈에 없으면 이름 주입(lookup 경로에 따라 먹을 수 있음)
    ar.__dict__["calculate_similarity"] = _cos_sim
    print("✅ injected ar.calculate_similarity")

    # 3) 클래스 메서드로 계산하는 버전 대비: ResponseRelevancy.calculate_similarity를 덮어쓰기
    if hasattr(ar, "ResponseRelevancy"):
        try:
            ar.ResponseRelevancy.calculate_similarity = _cos_sim
            print("✅ patched ResponseRelevancy.calculate_similarity")
        except Exception:
            pass

# Upstage 제약: n=1 강제
if hasattr(AR_METRIC, "strictness"):
    AR_METRIC.strictness = 1
    print("✅ AR_METRIC.strictness fixed to 1")

# -----------------------------
# 공통 유틸
# -----------------------------
def clip_rows(rows, *, max_ctx=4, max_chars=1200, clip_answer_chars=700):
    """요구사항 반영: context clip 1200 chars"""
    clipped = []
    for r in rows:
        rr = dict(r)
        ctx = rr.get("contexts", []) or []
        ctx = [c for c in ctx if c is not None]
        rr["contexts"] = [str(c)[:max_chars] for c in ctx[:max_ctx]]
        if rr.get("answer"):
            rr["answer"] = str(rr["answer"])[:clip_answer_chars]
        clipped.append(rr)
    return clipped

def cleanup_memory():
    gc.collect()
    print("🧹 Memory cleaned")

def save_df(df: pd.DataFrame, out_path: str):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    df.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✅ saved: {out_path}")
    return out_path

# -----------------------------
# 정성 체크(평가표 컬럼): 근거표현/문장구조/추론확장/불확실성
# - 엄밀한 metric이 아니라 "패턴 탐지용" 보조 컬럼
# -----------------------------
_KR_EVIDENCE_PHRASES = ["따르면", "근거", "참고", "에 따르면", "조항", "규정", "판례", "법원", "사건번호", "제"]
_KR_UNCERTAINTY_PHRASES = ["확인 필요", "추가 확인", "가능", "일 수", "자료 범위", "제공된 자료", "단정", "불명확"]

# ✅ (수정) 추론 신호를 더 넓게 잡기
# - 너무 일반적인 접속사는 과검출 위험이 있어, "논리 전개/요약/원인/조건" 중심으로 확장
_KR_INFERENCE_PHRASES = [
    "일반적으로", "통상", "보통", "대개", "추정", "원칙적으로", "추론", "상식적으로",
    "따라서", "그러므로", "그래서", "즉", "정리하면", "요약하면", "결론적으로",
    "왜냐하면", "때문에", "반대로", "만약", "경우", "전제", "가정",
    "→", "=>", "결과적으로", "종합하면",
]

def _has_any(text: str, phrases: List[str]) -> int:
    t = text or ""
    return int(any(p in t for p in phrases))

# -----------------------------
# ✅ (수정) 첫 문장 추출: 헤더(###/** 등) 스킵
# -----------------------------
_MD_HEADING_RE = re.compile(r"^\s{0,3}#{1,6}\s+")
_MD_BOLD_TITLE_RE = re.compile(r"^\s*\*\*.+\*\*\s*$")
_MD_LIST_TITLE_RE = re.compile(r"^\s*[-*•]\s+\S+")
_ONLY_PUNCT_RE = re.compile(r"^[\s\-\*\#\=\_]+$")

def _strip_md_bold(s: str) -> str:
    # ** ... ** 바깥 표시 제거
    t = s.strip()
    if t.startswith("**") and t.endswith("**") and len(t) >= 4:
        t = t[2:-2].strip()
    return t

def _looks_like_sentence(s: str) -> bool:
    # 문장처럼 보이는 힌트: 종결/구두점/길이
    if not s:
        return False
    # 따옴표 제거 후 체크
    t = s.strip().strip('"').strip("'").strip("“”").strip()
    if any(p in t for p in [".", "?", "!", "…"]):
        return True
    if any(t.endswith(end) for end in ["합니다", "됩니다", "있습니다", "없습니다", "하세요", "권합니다", "가능합니다", "불가능합니다", "입니다"]):
        return True
    # 너무 짧으면 제목일 확률이 높음
    return len(t) >= 18

def _first_meaningful_line(text: str) -> str:
    if not text:
        return ""
    lines = text.splitlines()
    for line in lines:
        s = (line or "").strip()
        if not s:
            continue
        if _ONLY_PUNCT_RE.match(s):
            continue
        if _MD_HEADING_RE.match(s):
            continue

        # ✅ (핵심 수정) 굵은 글씨 라인은 무조건 스킵하지 않는다
        if _MD_BOLD_TITLE_RE.match(s):
            inner = _strip_md_bold(s)
            # inner가 실제 문장(결론)처럼 보이면 채택
            if _looks_like_sentence(inner):
                return inner
            # 아니면 섹션 타이틀로 보고 스킵
            continue

        if _MD_LIST_TITLE_RE.match(s) and any(k in s for k in ["결론", "요약", "정리", "근거", "체크리스트", "절차", "증빙", "추가 확인", "확인 질문"]):
            continue

        return s
    return ""


def _first_sentence(text: str) -> str:
    if not text:
        return ""
    line = _first_meaningful_line(text)
    if not line:
        return ""
    # 문장 단위로 자르기
    for sep in [".", "?", "!"]:
        if sep in line:
            return line.split(sep, 1)[0].strip()
    return line.strip()

_CONCLUSION_HEADER_RE = re.compile(r"(한\s*줄\s*결론|^A\.\s*한\s*줄\s*결론|^A\.\s*결론|^결론)", re.IGNORECASE)

def _extract_conclusion_line(answer: str) -> str:
    if not answer:
        return ""
    lines = answer.splitlines()

    # 1) "한 줄 결론" 섹션을 찾고, 그 아래에서 첫 문장 후보를 뽑기
    for i, line in enumerate(lines):
        s = (line or "").strip()
        if not s:
            continue
        # 헤더 표식 제거 후에도 "한 줄 결론"인지 확인
        s2 = s.lstrip("#").strip()
        if _CONCLUSION_HEADER_RE.search(s2):
            # 아래 1~4줄에서 의미있는 문장 찾기
            for j in range(i + 1, min(i + 6, len(lines))):
                cand = (lines[j] or "").strip()
                if not cand:
                    continue
                # 굵게 감싸진 경우 내부만
                if cand.startswith("**") and cand.endswith("**") and len(cand) >= 4:
                    cand = cand[2:-2].strip()
                cand = cand.strip().strip('"').strip("'").strip("“”").strip()
                # 제목/구분선만 있으면 스킵
                if _MD_HEADING_RE.match(cand) or _ONLY_PUNCT_RE.match(cand):
                    continue
                return cand
            break

    return ""

def _looks_conclusion_first(answer: str) -> int:
    # 1순위: 한 줄 결론 섹션 기반
    c = _extract_conclusion_line(answer)
    if c:
        # 결론 줄이 너무 짧은 제목이면 제외
        if len(c) >= 8:
            return 1

    # 2순위: 기존 폴백
    s1 = _first_sentence(answer)
    if not s1:
        return 0

    t = s1.strip().strip('"').strip("'").strip("“”").strip()
    # 너무 짧으면 제목 가능성
    if len(t) < 8:
        return 0

    # 강한 종결/권고 신호
    if any(k in t for k in ["가능", "불가능", "필수", "중요", "권장", "추천", "위험", "유리", "불리"]):
        return 1
    if any(t.endswith(e) for e in ["합니다", "됩니다", "입니다", "해야 합니다", "필요합니다", "중요합니다"]):
        return 1

    return 0



def _norm_ws(s: str) -> str:
    # 공백/줄바꿈/따옴표 정리 + 다중 공백 축소
    if not s:
        return ""
    t = str(s)
    t = t.replace("“", '"').replace("”", '"').replace("’", "'")
    t = re.sub(r"\s+", " ", t).strip()
    return t

def _norm_article(art: str) -> str:
    # "제 30 조", "제30조", "30조" -> "제30조" 형태로 통일
    if not art:
        return ""
    t = re.sub(r"\s+", "", str(art))
    if re.match(r"^\d+조", t):  # "30조" 형태면
        t = "제" + t
    # "제3조의2"도 그대로 유지
    return t

def _hallucinated_citation_flag(answer: str, contexts: List[str]) -> int:
    """
    ✅ 개선판:
    - 답변에서 조문 토큰(제NN조/제NN조의M)을 뽑고
    - 컨텍스트에 같은 조문 토큰이 하나도 없으면 1(의심), 있으면 0
    - 공백 차이 때문에 오탐 나는 것 방지
    """
    if not answer:
        return 0

    ctx_all = _norm_ws("\n".join(contexts or []))
    if not ctx_all:
        # 컨텍스트가 비어있으면 '없는 조문'이라기보다 '비교 불가'에 가깝지만,
        # 기존 의미를 유지하려면 1로 두는 게 안전.
        return 1

    # 답변에서 조문만 추출 (law name까지 묶지 않음)
    arts = re.findall(r"(제?\s*\d+\s*조(?:\s*의\s*\d+)?)", answer)
    if not arts:
        return 0

    arts_norm = {_norm_article(a) for a in arts if _norm_article(a)}
    if not arts_norm:
        return 0

    ctx_nospace = _norm_article(ctx_all)  # 공백 제거한 버전으로도 비교
    # 하나라도 컨텍스트에 있으면 의심 아님
    for a in arts_norm:
        if a in ctx_all or a in ctx_nospace:
            return 0
    return 1


# -----------------------------
# 정규화 결과 “1줄 강제” 유틸
# (주의: 지금 패치는 trace 저장값만 1줄로 만들고, pipeline 내부 retrieval query에는 영향을 주지 않습니다.
#        pipeline 내부에 적용하려면 rag_module.py 쿼리 생성 직전에 동일 로직을 넣어야 합니다.)
# -----------------------------
def sanitize_normalized_query(text: str, max_chars: int = 400) -> str:
    if not text:
        return ""
    first = text.strip().splitlines()[0].strip()
    return first[:max_chars]

# ✅ 데이터 검증 함수 추가
def validate_data(rows, required_fields):
    """데이터 검증 및 문제 리포트"""
    issues = []
    for i, r in enumerate(rows):
        for field in required_fields:
            if field not in r:
                issues.append(f"Row {i}: missing field '{field}'")
            elif r[field] is None:
                issues.append(f"Row {i}: field '{field}' is None")
            elif field in ["question", "answer"] and not str(r[field]).strip():
                issues.append(f"Row {i}: field '{field}' is empty")

    if issues:
        print("⚠️ Data validation issues:")
        for issue in issues[:10]:  # 처음 10개만 출력
            print(f"  - {issue}")
        if len(issues) > 10:
            print(f"  ... and {len(issues) - 10} more issues")
    else:
        print("✅ Data validation passed")

    return len(issues) == 0

# -----------------------------
# 추가 컬럼 계산 함수
# - ✅ (요구 반영) 확인질문 로직 삭제: has_questions_2plus 관련 모두 제거
# -----------------------------
_RE_ARTICLE = re.compile(r"제\s*\d+\s*조(?:\s*의\s*\d+)?")  # '제 30 조', '제3조의2' 변형 대응
_RE_BULLET = re.compile(r"(^|\n)\s*[-*•]\s+")

def _to_context_list(x):
    """contexts를 list[str]로 정규화."""
    if x is None:
        return []
    if isinstance(x, list):
        return [str(s) for s in x if s is not None and str(s).strip()]
    s = str(x).strip()
    if not s:
        return []
    parts = [p.strip() for p in re.split(r"\n\s*\n", s) if p.strip()]
    return parts if parts else [s]

# -----------------------------
# 실행 태그 / 디렉토리
# -----------------------------
def make_run_tag_for_answer(model_name: str) -> str:
    return f"llmcmp_{model_name}"

RUNS_ROOT = os.path.join(OUT_DIR, "runs_answer")
os.makedirs(RUNS_ROOT, exist_ok=True)

def add_prompt_eval_columns(
    df: pd.DataFrame,
    answer_col: str = "answer",
    contexts_col: str = "contexts"
) -> pd.DataFrame:
    """
    df에 아래 컬럼 추가 (확인질문 로직 삭제 반영):
    - has_todo_checklist
    - has_legal_basis
    - citation_count_like
    - n_contexts
    - ctx_total_chars
    """
    df = df.copy()

    answers = df[answer_col].fillna("").astype(str) if answer_col in df.columns else pd.Series([""] * len(df))
    ctxs = df[contexts_col] if contexts_col in df.columns else pd.Series([[]] * len(df))

    # 1) 지금 당장 할 일/체크리스트 존재 여부 (섹션 키워드 OR 불릿 2개 이상)
    def has_todo(a: str) -> int:
        if not a:
            return 0
        key = ("지금 당장" in a) or ("할 일" in a) or ("체크리스트" in a)
        bullets = len(_RE_BULLET.findall(a)) >= 2
        return int(key or bullets)

    # 2) 법적 근거 섹션 존재 여부 (키워드 + 조문 패턴 1개 이상)
    def has_basis(a: str) -> int:
        if not a:
            return 0
        key = ("법적 근거" in a) or ("근거" in a) or ("조문" in a)
        art = len(_RE_ARTICLE.findall(a)) >= 1
        return int(key and art)

    # 3) 조문 인용 패턴 개수(유사 인용 개수)
    def citation_like(a: str) -> int:
        if not a:
            return 0
        return len(_RE_ARTICLE.findall(a))

    # 4) 컨텍스트 개수 / 5) 컨텍스트 총 길이
    n_contexts = []
    ctx_chars = []
    for x in ctxs:
        lst = _to_context_list(x)
        n_contexts.append(len(lst))
        ctx_chars.append(sum(len(s) for s in lst))

    df["has_todo_checklist"] = [has_todo(a) for a in answers]
    df["has_legal_basis"] = [has_basis(a) for a in answers]
    df["citation_count_like"] = [citation_like(a) for a in answers]
    df["n_contexts"] = n_contexts
    df["ctx_total_chars"] = ctx_chars

    return df

# -----------------------------
# run_once_answer: AnswerRelevancy / Faithfulness 1회
# -----------------------------
def run_once_answer(
    pipeline,
    items: List[Dict[str, Any]],
    run_idx: int,
    llm_name: str = "solar-pro2",
) -> Dict[str, Any]:
    run_tag = make_run_tag_for_answer(llm_name)
    run_dir = os.path.join(RUNS_ROOT, run_tag, f"run_{run_idx:02d}")

    os.makedirs(run_dir, exist_ok=True)

    # prompt_tag = PROMPT_TAG
    # with open(os.path.join(run_dir, "prompt_tag.txt"), "w", encoding="utf-8") as f:
    #     f.write(prompt_tag + "\n")

    cache_jsonl = os.path.join(run_dir, "rag_run_cache.jsonl")
    print(f"--- run_once_answer START | run_idx={run_idx} | llm_name={llm_name} ---")

    # (1) 파이프라인 실행 결과 캐시 저장
    with open(cache_jsonl, "w", encoding="utf-8") as f:
        print(f"items={len(items)}")
        for i, ex in enumerate(items, start=1):
            q = ex["question"]
            gt = ex.get("ground_truth", "")

            MAX_QUERY_CHARS = 2000
            q_for_retrieval = q if len(q) <= MAX_QUERY_CHARS else q[:MAX_QUERY_CHARS]

            try:
                trace = pipeline.answer_with_trace(q_for_retrieval, skip_normalization=False)
                if "normalized_query" in trace:
                    trace["normalized_query"] = sanitize_normalized_query(trace.get("normalized_query", ""))
            except Exception as e:
                trace = {"answer": "", "docs": [], "normalized_query": ""}

            answer = trace.get("answer", "") or ""
            docs = trace.get("docs", []) or []

            contexts = []
            for d in docs:
                try:
                    contexts.append(d.page_content)
                except Exception:
                    pass

            row = {
                "id": i,
                "question": q,
                "answer": answer,
                "contexts": contexts,
                "reference": gt,
                "ground_truths": [gt],
                "normalized_query": trace.get("normalized_query", ""),
            }
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

    # (2) rows 로드 + 최소 정리
    raw_rows = []
    with open(cache_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                raw_rows.append(json.loads(line))

    rows_local = []
    for r in raw_rows:
        rr = dict(r)
        if not rr.get("question"):
            continue
        if not rr.get("answer"):
            rr["answer"] = "답변을 생성할 수 없습니다."
        if "contexts" not in rr or rr["contexts"] is None:
            rr["contexts"] = []
        if not isinstance(rr["contexts"], list):
            rr["contexts"] = [str(rr["contexts"])]
        rows_local.append(rr)

    validate_data(rows_local, required_fields=["question", "answer", "contexts"])

    # (3) clip (요구사항: max_chars=1200)
    rows_eval = clip_rows(rows_local, max_ctx=4, max_chars=1200, clip_answer_chars=700)

    ds = Dataset.from_list(
        [
            {
                "question": r["question"],
                "answer": r["answer"],
                "contexts": r.get("contexts", []),
                "reference": r.get("reference", ""),
            }
            for r in rows_eval
        ]
    )

    # (4) RAGAS 평가
    print(f"evaluate START | run_idx={run_idx}")
    import sys, ragas.metrics._answer_relevance as ar

    print("ar module id:", id(ar))
    print("calc filename:", ar.calculate_similarity.__code__.co_filename)
    print("in sys.modules:", sys.modules.get("ragas.metrics._answer_relevance") is ar)

    AR_METRIC.strictness = 1
    res = evaluate(
        dataset=ds,
        metrics=[AR_METRIC, F_METRIC],
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        run_config=rc,
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )

    print("evaluate RETURNED")
    df = res.to_pandas()
    print("to_pandas DONE")

    # id 컬럼 보정
    if "id" not in df.columns:
        df = df.copy()
        df["id"] = range(1, len(df) + 1)

    # (5) 정성 체크 컬럼 추가
    id_to_ctx = {i + 1: rows_eval[i].get("contexts", []) for i in range(len(rows_eval))}
    id_to_ans = {i + 1: rows_eval[i].get("answer", "") for i in range(len(rows_eval))}

    # ✅ RAGAS 결과 df에는 answer/contexts가 없으므로 rows_eval에서 다시 붙임
    df["answer"] = df["id"].map(lambda _id: id_to_ans.get(_id, ""))
    df["contexts"] = df["id"].map(lambda _id: id_to_ctx.get(_id, []))

    df["evidence_expression"] = df["id"].map(lambda _id: _has_any(id_to_ans.get(_id, ""), _KR_EVIDENCE_PHRASES))
    df["structure_conclusion_first"] = df["id"].map(lambda _id: _looks_conclusion_first(id_to_ans.get(_id, "")))
    df["inference_expansion_signal"] = df["id"].map(lambda _id: _has_any(id_to_ans.get(_id, ""), _KR_INFERENCE_PHRASES))
    df["uncertainty_signal"] = df["id"].map(lambda _id: _has_any(id_to_ans.get(_id, ""), _KR_UNCERTAINTY_PHRASES))
    df["hallucinated_citation_suspect"] = df["id"].map(
        lambda _id: _hallucinated_citation_flag(id_to_ans.get(_id, ""), id_to_ctx.get(_id, []))
    )

    df = add_prompt_eval_columns(df, answer_col="answer", contexts_col="contexts")

    # 컬럼명 한글로 변경 (보기 편하게)
    RENAME_COLS_KR = {
        "evidence_expression": "근거표현유무",
        "structure_conclusion_first": "두괄식",
        "inference_expansion_signal": "추가추론유무",
        "uncertainty_signal": "안전표현유무",
        "hallucinated_citation_suspect": "없는조문의심",
        "has_todo_checklist": "체크리스트",
        "has_legal_basis": "법적근거섹션",
        "citation_count_like": "답변내조문갯수",
        "n_contexts": "컨텍스트개수",
        "ctx_total_chars": "컨텍스트길이",
    }
    df = df.rename(columns=RENAME_COLS_KR)

    # (저장1) ragas raw
    out_raw = os.path.join(run_dir, "ragas_result_answer_faith.csv")
    save_df(df, out_raw)

    # (저장2) per_question (보기 좋은 평가표)
    perq_cols = [
        "id",
        "answer",
        "contexts",
        "answer_relevancy",
        "faithfulness",
        "근거표현유무",
        "두괄식",
        "추가추론유무",
        "안전표현유무",
        "없는조문의심",
        "체크리스트",
        "법적근거섹션",
        "답변내조문갯수",
        "컨텍스트개수",
        "컨텍스트길이",
    ]

    # ✅ 실제 df에 존재하는 컬럼만 선택 (metrics를 바꿔도 KeyError 방지)
    _cols_exist = [c for c in perq_cols if c in df.columns]
    perq = df[_cols_exist].copy()

    for c in ["answer_relevancy", "faithfulness"]:
        if c in perq.columns:
            perq[c] = perq[c].astype(float).round(3)

    # 마지막 행에 전체 평균(숫자 컬럼만 평균)
    numeric_cols = perq.select_dtypes(include=["number"]).columns.tolist()
    mean_row = {c: perq[c].mean() for c in numeric_cols}
    mean_row["id"] = "mean"
    perq = pd.concat([perq, pd.DataFrame([mean_row])], ignore_index=True)

    perq_csv = os.path.join(run_dir, "per_question.csv")
    save_df(perq, perq_csv)

    print(f"saved per_question_csv={perq_csv}")
    print(f"--- run_once_answer DONE | run_idx={run_idx} ---")
    cleanup_memory()

    # mean 행 제외한 통계
    perq_metrics_only = perq[perq["id"] != "mean"].copy()

    return {
        "run_tag": run_tag,
        "run_idx": run_idx,
        "run_dir": run_dir,
        "cache_jsonl": cache_jsonl,
        "raw_csv": out_raw,
        "per_question_csv": perq_csv,
        "AR_mean": float(perq_metrics_only["answer_relevancy"].mean()),
        "AR_std": float(perq_metrics_only["answer_relevancy"].std(ddof=1)) if len(perq_metrics_only) > 1 else 0.0,
        "F_mean": float(perq_metrics_only["faithfulness"].mean()),
        "F_std": float(perq_metrics_only["faithfulness"].std(ddof=1)) if len(perq_metrics_only) > 1 else 0.0,
    }

# -----------------------------
# 반복 실행(run_repeat_answer) + wide CSV
# -----------------------------
def run_repeat_answer(pipeline, items: List[Dict[str, Any]], n: int = 3, llm_name: str = "solar-pro2") -> List[Dict[str, Any]]:
    results = []
    for run_idx in range(1, n + 1):
        print("\n==============================")
        print(f"RUN {run_idx:02d}/{n} | {make_run_tag_for_answer(llm_name)}")
        print("==============================")
        results.append(run_once_answer(pipeline, items, run_idx, llm_name=llm_name))
    return results

def build_wide_csv_answer(run_tag: str, results: List[Dict[str, Any]], n_questions: int) -> str:
    """
    메모리 안정성을 위해 결과 dict에 df_perq를 저장하지 않고,
    per_question_csv를 다시 읽어서 wide를 만듭니다.
    """
    base_dir = os.path.join(RUNS_ROOT, run_tag)
    os.makedirs(base_dir, exist_ok=True)

    ids = pd.DataFrame({"id": range(1, n_questions + 1)})
    wide = ids.copy()

    for res in results:
        ridx = res["run_idx"]
        dfm = pd.read_csv(res["per_question_csv"])[[
            "id",
            "answer_relevancy",
            "faithfulness",
            "없는조문의심",  # ✅ 한글 컬럼명으로 변경 반영
        ]].copy()

        dfm = dfm.rename(columns={
            "answer_relevancy": f"r{ridx}-answer_relevancy",
            "faithfulness": f"r{ridx}-faithfulness",
            "없는조문의심": f"r{ridx}-없는조문의심",
        })
        wide = pd.merge(wide, dfm, on="id", how="left")

    wide = wide.set_index("id").sort_index()

    ar_cols = [c for c in wide.columns if c.endswith("-answer_relevancy")]
    f_cols = [c for c in wide.columns if c.endswith("-faithfulness")]

    wide["answer_relevancy-mean"] = wide[ar_cols].mean(axis=1, skipna=True)
    wide["faithfulness-mean"] = wide[f_cols].mean(axis=1, skipna=True)

    mean_row = wide.mean(axis=0, skipna=True).to_frame().T
    mean_row.index = ["mean"]
    wide2 = pd.concat([wide, mean_row], axis=0)

    out_csv = os.path.join(base_dir, "runs_wide_3repeats.csv")
    wide2.to_csv(out_csv, encoding="utf-8")
    print("✅ saved:", out_csv)
    return out_csv

# -----------------------------
# DOCX 질문 로딩
# -----------------------------
from docx import Document

DOCX_PATH = "C:\ai\source\chatbot_app\ragas_testset_10_selected.jsonl"

def load_items_from_docx(docx_path: str, max_q: int = 10):
    doc = Document(docx_path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]

    items = []
    i = 0
    while i < len(paras) and len(items) < max_q:
        line = paras[i]
        m = re.match(r"^(\d+)\.\s*(.+)$", line)
        if not m:
            i += 1
            continue

        question = m.group(2).strip()

        i += 1
        answer_lines = []
        while i < len(paras):
            if re.match(r"^\d+\.\s*.+$", paras[i]):
                break
            answer_lines.append(paras[i])
            i += 1

        gt = "\n".join([x for x in answer_lines if x not in ["✔️ 모범답안"]]).strip()
        items.append({"question": question, "ground_truth": gt})

    return items

items = load_items_from_docx(DOCX_PATH, max_q=10)
print("✅ items loaded:", len(items))
print("sample Q1:", items[0]["question"])

<>:65: SyntaxWarning: invalid escape sequence '\s'
<>:65: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_16380\525142971.py:65: SyntaxWarning: invalid escape sequence '\s'
  OUT_DIR = "C:\ai\source\chatbot_app"
C:\ai\source\chatbot_app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Admin\AppData\Local\Temp\ipykernel_16380\525142971.py:21: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import answer_relevancy as AnswerRelevancy
C:\Users\Admin\AppData\Local\Temp\ipykernel_16380\525142971.py:65: SyntaxWarning: invalid escape sequence '\s'
  OUT_DIR = "C:\ai\source\cha

ModuleNotFoundError: No module named 'chatbot_app.modules'

# 2. 실행

In [ ]:
%%time

from langchain_openai import ChatOpenAI

gpt_llm = ChatOpenAI(
    # model="gpt-4o-mini",
    model="gpt-5-mini",
    temperature=0
)

upstage_llm = ChatUpstage(
    model="solar-pro2",
    temperature=0
)

pipeline = create_pipeline(
    # generation_llm=gpt_llm,
    generation_llm=upstage_llm,
    embedding=upstage_embedding,
)

# -----------------------------
# 1회 실행
# -----------------------------
result = run_once_answer(pipeline, items, run_idx=1, llm_name="solar-pro2")

# 저장 파일들 경로 출력
print("✅ DONE. run_tag =", result.get("run_tag"))
print(" - raw_csv:", result.get("raw_csv"))
print(" - run_dir:", result.get("run_dir"))
print(" - per_question_csv:", result.get("per_question_csv"))


2026-02-05 15:00:11,679 - chatbot_app.modules.rag_module - INFO - 🔗 Pinecone 3중 인덱스 연결 중...
2026-02-05 15:00:14,761 - chatbot_app.modules.rag_module - INFO - ✅ [Law / Rule / Case] 3개 인덱스 로드 완료!
2026-02-05 15:00:14,791 - chatbot_app.modules.rag_module - INFO - ℹ️ SimpleTokenizer 사용 (BM25)


--- run_once_answer START | run_idx=1 | llm_name=solar-pro2 ---
items=10


2026-02-05 15:00:15,727 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-05 15:00:15,727 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 전입신고(주민등록)·확정일자(확정일자) 했는데, 확정일자부(확정일자) 내용까지 중요한가요?
2026-02-05 15:00:15,727 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='전입신고(주민등록)·확정일자(확정일자) 했는데, 확정일자부(확정일자) 내용까지 중요한가요?'
2026-02-05 15:00:16,053 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-05 15:00:18,075 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-05 15:00:20,163 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-05 15:00:22,269 - httpx - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
2026-02-05 15:00:22,285 - chatbot_app.modules.rag_module - INFO - 📌 Rerank selected=2 (threshold=0.2)
2026-02-05 15:00:22,286

2026-02-05 15:00:57,789 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='조정 신청은 어디에 하나요? (분쟁조정) 말로 해도 되나요? (분쟁조정) 돈은 꼭 내야 하나요? (분쟁조정)  

※ 참고: 용어 사전 내 "조정위" → "주택임대차분쟁조정위원회" 매핑이 존재하나, 사용자 질문에 해당 단어가 없어 적용되지 않았습니다. "분쟁조정"은 사전 미등재 단어로 원안 유지되었습니다.'
2026-02-05 15:00:58,494 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-05 15:00:59,220 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-05 15:01:00,128 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-05 15:01:00,707 - httpx - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
2026-02-05 15:01:00,707 - chatbot_app.modules.rag_module - INFO - 📌 Rerank selected=7 (threshold=0.2)
2026-02-05 15:01:00,723 - chatbot_app.modules.rag_module - INFO - 🤖 답변 생성 중...
2026-02-05 15:01:03,951 - httpx - INFO - HTTP Request: POST https://api.upstag

✅ Data validation passed
evaluate START | run_idx=1
ar module id: 1732619184992
calc filename: C:\Users\Admin\AppData\Local\Temp\ipykernel_19848\2654277498.py
in sys.modules: True


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

Batch 1/20:   0%|          | 0/1 [00:00<?, ?it/s]

2026-02-05 15:01:27,351 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-05 15:01:28,620 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-05 15:01:29,324 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-05 15:01:32,400 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-05 15:01:43,980 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-05 15:01:45,512 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-05 15:01:46,771 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-05 15:01:47,485 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-

In [ ]:
df = pd.read_csv('data/RAGAS/runs_answer/llmcmp_solar-pro2/run_01/per_question.csv')
df